### Autograd

- 什么是导数        
    - 导数（一元函数）是变化率、是切线的斜率、是瞬时速度；
    - 可导与不可导      

![导数.png](导数.png)

- 什么是方向导数        
    - 函数在A点无数个切线的斜率的定义。每一个切线都代表一个变化的方向。     
![无数个切线.png](无数个切线.png)


- 什么是偏导数      
    - 多元函数降维时候的变化，比如：二元函数固定y，只让x单独变化，从而看成是关于x的一元函数的变化来研究     
    ![多元函数.png](多元函数.png)

- 什么是梯度
    - 函数在A点无数个变化方向中变化最快的那个方向。     
    - 梯度下降，就是沿着方向变化最快的方向去寻找最优解。        
![](梯度.png)


- 梯度与机器学习中的最优解      
- 有监督学习，无监督学习，半监督学习    
- 样本x，标签y

- 机器学习就是找到最小的loss
- loss, argmin loss，loss=| f(w,x)-y |^2    

### Variable is Tensor

- 目前Variable 已经与Tensor合并。       
- 每个tensor通过requires_grad来设置是否计算梯度。       
    - 用来冻结某些层的参数      

- 如何计算梯度
- 链式法则：两个函数组合起来的符合函数，导数等于里面函数带入外函数值的导乘以里面函数之导数。        
- 举个栗子： y = 2x, z=y*y, 求dz/dx。        

### 关于Autograd的几个概念

- 叶子张量（leaf)       
    ![](叶子张量.png)

- grad VS grad_fn       
    - grad: 该Tensor的梯度值，每次在计算backward时都需要将前一时刻的梯度归零，否则梯度值会一直累加；    
    - grad_fn：叶子节点通常为None, 只有结果节点的grad_fn才有效，用于指示梯度函数是哪种类型。        
    

- backward函数      
    - torch.autograd.backward(tensor, grad_tensors = None, retain_graph = None, create_graph=False)
        - tensor: 用于计算梯度的tensor, torch.autograd.backward(z) == z.backward()
        - grad_tensor：在计算矩阵的梯度时会用到。他其实也是一个tensor，shape一般需要和前面的tensor保持一致。    
        - retain_graph：通常在调用一次backward后，pytorch会自动把计算图销毁，所以要想对某个变量重复调用backward，则需要将该参数设置为True       
        - create_graph: 如果为RTrue, 那么就创建一个专门的graph of the derivaive，这可以方便计算高阶微分。       


- torch.autograd.grad()函数     
    - def grad(outputs, inputs, grad_outputs=None, retain_graph=None, create_graph=False, only_inputs=True, allow_unused=False)
        - 计算和返回output关于inputs的梯度的和。
        - outputs：函数的因变量，即需要求导的那个函数。     
        - inputs：函数额自变量，
        - grad_outputs：同backward      
        - only_inputs：只计算input的梯度        
        - allow_unused(bool，可选)：如果为False，当计算输出出错时（因此他们的梯度永远是0）指明不使用的inputs。      

- torch.autograd包中的其他函数      
    - torch.autograd.enable_grad：启动梯度计算的上下文管理器        
    - torch.autograd.no_grad：禁止梯度计算的上下文管理器        
    - torch.autograd.set_grad_enabled(mode): 设置是否进行梯度计算的上下文管理器。       
    

- torch.autograd.Function       
    - 每一个原始的自动求导运算实际上是两个在Tensor上的运行的函数        
        - forward函数计算从输入Tensor获得的输出Tensors      
        - backward函数接收输出Tensors对于某个标量值的梯度，并且计算输入Tensors相对于该相同标量值的梯度。        
        - 最后，利用apply方法执行相应的运算     
            - 定义在Function类的父类_FunctionBase中定义的一个方法       

In [1]:
import torch

class line(torch.autograd.Function):
    @staticmethod
    def forward(ctx, w, x, b):
        ctx.save_for_backward(w, x, b)
        return w*x+b
    
    @staticmethod
    def backward(ctx, grad_out):
        w, x, b = ctx.saved_tensors

        grad_w = grad_out*x
        grad_x = grad_out*w
        grad_b = grad_out

        return grad_w, grad_x, grad_b



In [2]:
w = torch.rand(2, 2, requires_grad=True)
x = torch.rand(2, 2, requires_grad=True)
b = torch.rand(2, 2, requires_grad=True)

In [3]:
out = line.apply(w, x, b)
out.backward(torch.ones(2, 2))

print(w, x, b)
print(w.grad, x.grad, b.grad)

tensor([[0.2499, 0.3156],
        [0.4506, 0.3886]], requires_grad=True) tensor([[0.1759, 0.3622],
        [0.5290, 0.3648]], requires_grad=True) tensor([[0.3168, 0.9686],
        [0.2804, 0.8752]], requires_grad=True)
tensor([[0.1759, 0.3622],
        [0.5290, 0.3648]]) tensor([[0.2499, 0.3156],
        [0.4506, 0.3886]]) tensor([[1., 1.],
        [1., 1.]])
